<a href="https://colab.research.google.com/github/mwaoh/Pytorch/blob/main/Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
from google.colab import drive
drive.mount('')

#from google.colab import drive
#uploades = files.upload()
#df = pd.read_csv("zomato.csv", encoding = "latin-1")

# Read data from file
data = pd.read_csv("data.csv")

# Print the first 5 rows of the data
print(data.head())

# Visualize data with a scatter plot
plt.scatter(data['Feature1'], data['Feature2'])
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Scatter plot of Feature 1 vs. Feature 2')
plt.show()

In [ ]:
# Sample data
data = np.array([['apple', 'red', 'round'],
                 ['banana', 'yellow', 'curved'],
                 ['orange', 'orange', 'round']])

# One-hot encoding
vocab = np.unique(data) # Create vocabulary of unique values
onehot = np.zeros((data.shape[0], len(vocab))) # Initialize one-hot matrix
for i, row in enumerate(data):
    for j, item in enumerate(row):
        onehot[i, np.where(vocab == item)[0]] = 1 # Set one-hot value to 1 for matching vocabulary index
print("One-hot matrix:\n", onehot)

# Embedding
embedding_dim = 3 # Embedding dimension
embedding = nn.Embedding(len(vocab), embedding_dim) # Initialize embedding layer
inputs = torch.tensor([np.where(vocab == item)[0][0] for item in data.flatten()], dtype=torch.long) # Convert data to indices
inputs = inputs.view(data.shape[0], -1) # Reshape indices to match original data shape
embedded = embedding(inputs) # Embed inputs
print("Embedded matrix:\n", embedded.detach().numpy())


In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input):
        embedded = self.embedding(input)
        output, hidden = self.lstm(embedded)
        return output, hidden

class Decoder(nn.Module):
    def __init__(self, output_size, hidden_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        embedded = self.embedding(input)
        output, hidden = self.lstm(embedded, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input, target):
        enc_output, enc_hidden = self.encoder(input)
        dec_hidden = enc_hidden
        dec_input = torch.tensor([[SOS_token]], device=device)
        dec_outputs = torch.zeros(target.size(0), target.size(1), self.decoder.output_size, device=device)
        for t in range(target.size(1)):
            dec_output, dec_hidden = self.decoder(dec_input, dec_hidden)
            dec_outputs[:, t, :] = dec_output
            dec_input = target[:, t].unsqueeze(1)
        return dec_outputs


In [ ]:
# Define your GAN architecture using PyTorch
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        # Define your generator architecture using PyTorch layers
    
    def forward(self, x):
        # Define the forward pass for your generator

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        # Define your discriminator architecture using PyTorch layers
    
    def forward(self, x):
        # Define the forward pass for your discriminator

# Define your loss function and optimizer
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Define your training data and data loader
class POIDataset(Dataset):
    def __init__(self):
        # Load and preprocess your POI data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        # Return a sample from your POI data
    
dataset = POIDataset()
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Train your GAN on your POI data
for epoch in range(num_epochs):
    for i, data in enumerate(dataloader):
        # Train your generator and discriminator using PyTorch
        # Update the weights of your networks using the optimizer
    
# Use your trained GAN to generate new POI information
location = [


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define the discriminator network
class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Discriminator, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.linear(out[:, -1, :])
        out = torch.sigmoid(out)
        return out

# Define the generator network
class Generator(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(Generator, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.linear(out)
        return out

# Define the loss function
def gail_loss(expert_seq, generated_seq, discriminator):
    expert_labels = torch.ones(expert_seq.shape[0], 1).cuda()
    generated_labels = torch.zeros(generated_seq.shape[0], 1).cuda()
    expert_preds = discriminator(expert_seq)
    generated_preds = discriminator(generated_seq)
    adversarial_loss = nn.BCELoss()(generated_preds, expert_labels)
    imitation_loss = nn.MSELoss()(generated_seq, expert_seq)
    total_loss = adversarial_loss + imitation_loss
    return total_loss

# Define the hyperparameters
input_size = 10
output_size = 5
hidden_size = 64
batch_size = 32
num_epochs = 1000
learning_rate = 0.001

# Initialize the discriminator and generator networks
discriminator = Discriminator(input_size + output_size, hidden_size).cuda()
generator = Generator(input_size, output_size, hidden_size).cuda()

# Define the optimizer and learning rate scheduler
discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=learning_rate)
generator_optimizer = optim.Adam(generator.parameters


In [ ]:
# Load the necessary libraries and data
import numpy as np
from keras.models import load_model
from preprocess import load_data

# Load the trained generator model
generator = load_model('generator_model.h5')

# Load the data used to train the GAN
data = load_data('poi_data.csv')

# Define the size of the latent space vector used to generate new POI information
latent_dim = 100

# Generate a random vector in the latent space
location = np.random.randn(1, latent_dim)

# Use the generator model to generate new POI information from the random vector
generated_poi_info = generator.predict(location)

# Print the generated POI information
print(generated_poi_info)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define the LSTM-based sequence generator
class LSTMGenerator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMGenerator, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        output, hidden = self.lstm(input, hidden)
        output = self.out(output.view(-1, self.hidden_size))
        return output, hidden
    
    def init_hidden(self, batch_size):
        return (torch.zeros(1, batch_size, self.hidden_size),
                torch.zeros(1, batch_size, self.hidden_size))

# Define the CNN-based discriminator
class CNNDiscriminator(nn.Module):
    def __init__(self, vocab_size, embedding_size, filter_sizes, num_filters):
        super(CNNDiscriminator, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_filters, (f, embedding_size)) for f in filter_sizes
        ])
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(num_filters * len(filter_sizes), 1)
    
    def forward(self, input):
        input = self.embedding(input)
        input = input.unsqueeze(1)
        conv_outputs = []
        for conv in self.convs:
            conv_output = conv(input)
            conv_output = nn.functional.relu(conv_output.squeeze(3))
            pool_output = nn.functional.max_pool1d(conv_output, conv_output.size(2)).squeeze(2)
            conv_outputs.append(pool_output)
        output = torch.cat(conv_outputs, 1)
        output = self.dropout(output)
        output = self.fc(output)
        return output.squeeze()

# Define the SeqGAN model
class SeqGAN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, vocab_size, embedding_size, filter_sizes, num_filters, learning_rate):
        super(SeqGAN, self).__init__()
        self.generator = LSTMGenerator(input_size, hidden_size, output_size)
        self.discriminator = CNNDiscriminator(vocab_size, embedding_size, filter_sizes, num_filters)
        self.optimizer_G = optim.Adam(self.generator.parameters(), lr=learning_rate)
        self.optimizer_D = optim.Adam(self.discriminator.parameters(), lr=learning_rate)
        self.criterion = nn.BCEWithLogitsLoss()
    
    def generate(self, input, length):
        batch_size = input.size(1)
        hidden = self.generator.init_hidden(batch_size)
        output_seq = []
        for i in range(length):
            output, hidden = self.generator(input, hidden)
            output_seq.append(output)
            input = output.detach()
        output_seq = torch.cat(output_seq, dim=0)
        return output_seq
    
    def train_G(self, input, target, reward):
        batch_size = input.size(1)
        hidden = self.generator.init_hidden(batch_size)
        output_seq = []
        for i in range(target.size(0)):
            output, hidden = self.generator(input, hidden)
            output_seq.append(output)
            input = target[i].unsqueeze(0)
        output_seq = torch.cat(output_seq, dim=0)
        loss = torch.sum(-reward * output_seq) / batch_size
        self.optimizer_G.zero_grad()
        loss.backward()
        self.optimizer_G.step()
    
    def
